# Collaborative filters' comparison

In [1]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from source.models.base_model import baseModel
from source.models.collabFilter_UserUser import CollaborativeFilter_UserUser
from source.models.collabFilter_ItemItem import CollaborativeFilter_ItemItem
# from source.models.SVD import SVDModel

import source.utils.utils as utils

In [2]:
data = pd.read_csv('../data/data_clean.csv')
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/test_data.csv')

In [3]:
U = train_data.pivot_table(index='tessera', columns='liv3', values='value') # or index='id_sc'

In [4]:
U

liv3,10101,10102,10103,10104,10105,10106,10107,10108,10201,10202,...,550104,550105,550106,550108,560112,570114,570213,570214,910101,999999
tessera,,,,,,,,,,,,,,,,,,,,,
2.484012e+11,NaN,1.0,1.181818,1.272727,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.272727
2.484015e+11,1.090909,1.0,NaN,1.181818,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2.484029e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.484041e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.484042e+11,NaN,NaN,NaN,1.272727,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4.330002e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.330002e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.330002e+11,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cfUser : baseModel =  CollaborativeFilter_UserUser(N = 100)
cfUser.fit(U)

In [6]:
cfItem : baseModel =  CollaborativeFilter_ItemItem(N = 100)
cfItem.fit(U)

In [7]:
# cfSVD : baseModel =  SVDModel(k = 2)
# cfSVD.fit(U)

In [8]:
# predicted_ratings_user, predicted_ratings_item = utils.parallel_predict(cfUser, cfItem, test_data)
predicted_ratings_item = utils.predict(cfItem, test_data)
# preticted_ratings_svd = utils.predict(cfSVD, test_data)

  0%|                                                                                                                                                                                 | 0/40800 [00:00<?, ?it/s]/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_ItemItem.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))
  1%|█▊                                                                                                                                                                    | 439/40800 [00:02<03:55, 171.33it/s]/Users/daniele/Documents/Università/terzoAnno/primoSemestre/SocialMediaDataAnalysis/Progetto/ShopSmart/source/models/collabFilter_ItemItem.py:21: RuntimeWarning: invalid value encountered in scalar divide
  predicted_rating = (self.U.loc[xi, similar_items] * selected_similarities).sum() / selected_similar

In [9]:
# mae_score_user = utils.mae(test_data['value'], predicted_ratings_user)
mae_score_item = utils.mae(test_data['value'], predicted_ratings_item)
# mae_score_svd = utils.mae(test_data['value'], preticted_ratings_svd)

In [10]:
k = 100

In [11]:
# precision_user = utils.precision_at_k(test_data['value'], predicted_ratings_user, k)
precision_item = utils.precision_at_k(test_data['value'], predicted_ratings_item, k)
# precision_svd = utils.precision_at_k(test_data['value'], preticted_ratings_svd, k)


In [12]:
# recall_user = utils.recall_at_k(test_data['value'], predicted_ratings_user, k)
recall_item = utils.recall_at_k(test_data['value'], predicted_ratings_item, k)
# recall_svd = utils.recall_at_k(test_data['value'], preticted_ratings_svd, k)

In [13]:
# f1_user = utils.f1_score_at_k(test_data['value'], predicted_ratings_user, k)
f1_item = utils.f1_score_at_k(test_data['value'], predicted_ratings_item, k)
# f1_svd = utils.f1_score_at_k(test_data['value'], preticted_ratings_svd, k)

In [14]:
# display(f"Precision User: {precision_user}")
# display(f"Recall User: {recall_user}")
# display(f"F1 User: {f1_user}")

display(f"Precision Item: {precision_item}")
display(f"Recall Item: {recall_item}")
display(f"F1 Item: {f1_item}")

# display(f"Precision SVD: {precision_svd}")
# display(f"Recall SVD: {recall_svd}")
# display(f"F1 SVD: {f1_svd}")

'Precision Item: 0.03'

'Recall Item: 7.352941176470588e-05'

'F1 Item: 0.00014669926650366747'

In [15]:
xi = U.index[0] # for example
print(f"Item-Item Collaborative Filtering has a better performance with a MAE of {mae_score_item}")
recommendations = cfItem.recommend(xi, 5)
# print(f"SVD Collaborative Filtering has a better performance with a MAE of {mae_score_svd}")
# recommendations = cfSVD.recommend(xi, 5)

# if mae_score_user < mae_score_item:
#     print(f"User-User Collaborative Filtering has a better performance with a MAE of {mae_score_user}")
#     recommendations = cfUser.recommend(xi, k)
# else:
#     print(f"Item-Item Collaborative Filtering has a better performance with a MAE of {mae_score_item}")
#     recommendations = cfItem.recommend(xi, k)

Item-Item Collaborative Filtering has a better performance with a MAE of 0.2563663712899397


In [16]:
data['liv3'] = data['liv3'].astype(str)

In [17]:
descriptions = []
for item, _ in recommendations:
    item_str = str(item)
    if not data.loc[data['liv3'] == item_str, 'descr_liv3'].empty:
        descriptions.append(data.loc[data['liv3'] == item_str, 'descr_liv3'].values[0])
    else:
        print(f"Item {item_str} non trovato nel DataFrame")

recommendations_str = "\n".join(descriptions)
print(f"Top 5 recommendations for user {xi}:\n{recommendations_str}")


Top 5 recommendations for user 248401194304.0:
CEREALI E FARINE INFANZIA
ALTRI FUORI PASTO SALATI
PRONTI DA GELARE
DOLCIFICANTI
ALTRE FORME DI CIOCC/ZUCCH
